In [42]:
from helpers import stats, weapon_types, prof_translator, item_translator, proffesions
from game_structures import Item

In [43]:
import csv

# write to file serialized items
def csvSerializer(items : list, item_type : str, output_file : str):
    with open(output_file, "w", encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['lvl', 'prof', 'type', 'name', 'stats'])
        for el in items:
            # Split each item's data
            split_lst = el['data-stat'].split(';')
            # return & split by only those which contain '='
            params = [p.split('=') for p in split_lst if "=" in p]
            dct = {p[0] : p[1] for p in params}
            # add key for lack of proffesion requirement
            if 'reqp' not in dct:
                dct['reqp'] = 'wpbhtm'
            try:
                single_item = Item(int(dct['lvl']),
                    dct['reqp'], 
                    item_type, 
                    el['data-name'], 
                    {stats[p] : dct[p] for p in dct if p in stats})
                #print(single_item.toList())
                writer.writerow(single_item.toList())  
            except KeyError:
                print(f"KeyError -> Item name: {el['data-name']}; stats:{el['data-stat']}")


In [44]:
import requests
from bs4 import BeautifulSoup
from time import sleep

# Blade dancer url:
#https://margohelp.pl/przedmioty/b/1/4/TancerzOstrzy/Jednor%C4%99czne/Heroiczne/1-150/
# Paladin url:
#https://margohelp.pl/przedmioty/p/1/4/Paladyn/Jednor%C4%99czne/Heroiczne/1-150/
# Warrior url:
#https://margohelp.pl/przedmioty/w/1/4/Wojownik/Jednor%C4%99czne/Heroiczne/1-150/
# Mage url:
#https://margohelp.pl/przedmioty/m/6/4/Mag/R%C3%B3%C5%BCd%C5%BCki/Heroiczne/1-150/
# Tracker url:
#https://margohelp.pl/przedmioty/t/4/4/Tropiciel/Dystansowe/Heroiczne/1-150/
# Hunter url:
#https://margohelp.pl/przedmioty/h/4/4/%C5%81owca/Dystansowe/Heroiczne/1-150/

# Function for downloading each proffesion with their item type items in separate .csv file
def scrapeAndSaveData():
    for key, vals in prof_translator.items():
        for idx, acc in enumerate(vals):
            page_items = []
            for i in (0, 150):
                sleep(.5)
                page = requests.get(f"https://margohelp.pl/przedmioty/{key[0]}/{acc}/4/{proffesions[key[0]]}/{item_translator[acc]}/Heroiczne/{1+i}-{150+i}/")
                soup = BeautifulSoup(page.content, 'html.parser')
                page_items.extend(soup.find_all(class_='item'))
            csvSerializer(page_items, weapon_types[key][idx], f'items/{proffesions[key[0]]}_{item_translator[acc]}.csv')

In [45]:
# Download items
scrapeAndSaveData()

KeyError related to: <div class="item" data-cl="8" data-icon="zbr/zbroja412.gif" data-id="51187" data-name="Peleryna wygnanego kapłana" data-pr="36" data-stat="binds;heroic;reqp=m;absorb=63;absorbm=31;ac=10;crit=1;critmval=6;di=5;resfrost=14;sa=9;"><img src="https://micc.garmory-cdn.cloud/obrazki/itemy/zbr/zbroja412.gif"/></div>
